In [2]:
from models.mc import MonteCarloBase, hybridMonteCarlo
import models.benchmarks as bm
from models.longstaff import LSMC_Numpy, LSMC_OpenCL
from models.pso import PSO_Numpy, PSO_OpenCL
from models.utils import checkOpenCL

import numpy as np
import matplotlib.pyplot as plt

In [3]:
checkOpenCL()


OpenCL Platforms and Devices
Platform - Name: Apple
Platform - Vendor: Apple
Platform - Version: OpenCL 1.2 (Dec 13 2024 23:09:21)
Platform - Profile: FULL_PROFILE
 --------------------------------------------------------
 Device - Name: Apple M3 Max
 Device - Type: ALL | GPU
 Device - Max Clock Speed: 1000 Mhz
 Device - Compute Units: 40
 Device - Global Memory: 48 GB
 Device - Constant Memory: 1 GB
 Device - Local Memory: 32 KB (LOCAL)
 Device - Max Buffer/Image Size: 9 GB
 Device - Max Work Group Size: 256




In [4]:
S0, r, sigma, T, nPath, nPeriod, K, opttype, nFish = 100.0, 0.03, 0.3, 1.0, 5000, 200, 115.0, 'P', 500
# S0, r, sigma, T, nPath, nPeriod, K, opttype, nFish = 100.0, 0.03, 0.3, 1.0, 50000, 200, 110.0, 'P', 500
mc = hybridMonteCarlo(S0, r, sigma, T, nPath, nPeriod, K, opttype, nFish)
mc.St

array([[101.989784, 101.69084 ,  99.34267 , ..., 133.52338 , 139.88774 ,
        139.86293 ],
       [ 98.37808 ,  99.696434, 100.30378 , ...,  56.089516,  56.16673 ,
         54.612133],
       [101.2071  , 102.34518 , 104.56201 , ...,  87.054855,  84.48822 ,
         85.74317 ],
       ...,
       [101.62027 , 103.51106 ,  99.449776, ...,  97.28607 ,  94.89542 ,
         95.33876 ],
       [ 99.443756,  96.016075,  98.36795 , ...,  84.803894,  83.4631  ,
         81.18077 ],
       [ 99.760635,  99.12108 ,  97.16456 , ...,  96.3229  ,  97.19287 ,
         92.94715 ]], dtype=float32)

In [5]:
# St_full = np.concatenate( (np.full(shape=(nPath, 1), fill_value=S0), mcBase.St) , axis=1)
# St_full
# plt.plot(St_full.T, color="b", lw=.7, alpha=.7)
# plt.show()

In [6]:
# benchmarks
bm.blackScholes(S0, K, r, sigma, T, opttype)
bm.binomialEuroOption(S0, K, r, sigma, nPeriod, T, opttype)
mc.getEuroOption_np()
mc.getEuroOption_cl()
bm.binomialAmericanOption(S0, K, r, sigma, nPeriod, T, opttype)

Black-Scholes price: 19.243573269491435
Binomial European price: 19.251305064768164
MonteCarlo Numpy European price: 19.3273125
MonteCarlo OpenCL_GPU European price: 19.3273078125
Binomial American price: 19.92214479009919


19.92214479009919

In [7]:
lsmc_np = LSMC_Numpy(mc, inverseType='benchmark_pinv')
C_hat, _, _ = lsmc_np.longstaff_schwartz_itm_path_fast()
C_hat

Longstaff numpy price: 19.84065492854219


19.84065492854219

In [8]:
lsmc_cl = LSMC_OpenCL(mc, inverseType='GJ')
C_hat, _, _ = lsmc_cl.longstaff_schwartz_itm_path_fast_hybrid()
C_hat

Longstaff OpenCL_GPU price: 20.050673423405133


20.050673423405133

In [9]:
pso_np = PSO_Numpy(mc, nFish, mc.costPsoAmerOption_np)
pso_np.solvePsoAmerOption_np()

Pso numpy price: 20.038209915161133


20.03821

In [10]:
pso_cl = PSO_OpenCL(mc, nFish, mc.costPsoAmerOption_cl)
pso_cl.solvePsoAmerOption_cl()

# clear up memory
pso_cl.cleanUp()
mc.cleanUp()

Pso OpenCL_GPU price: 20.038240432739258
